In [222]:
import pickle
import spacy
from spacy import displacy
import pandas as pd
import numpy as np
import contractions
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
import nltk

nlp = spacy.load("en_core_web_trf")
#only_uncleaned_dict=pickle.load(open("only_uncleaned_dict.pickle", "rb"))

#reading df
submissions_df=pickle.load(open("submissions_df.pickle", "rb"))
comments_df=pickle.load(open("comments_df.pickle", "rb"))




#filtering out comments or submissions with less than three votes
comments_df=comments_df[(comments_df['score']<=0)|(comments_df['score']>=2)]
submissions_df=submissions_df[(submissions_df['score']<=0)|(submissions_df['score']>=2)]
#merging comments and posts 
comments_df=comments_df['body'].append(submissions_df['title']).append(submissions_df['selftext'])
comments_df.name='body'

def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)

In [223]:

# remove urls up here so they're caught by the filter in next chunk
comments_df=comments_df.dropna().apply(remove_URL)


In [224]:

# save value counts
value_counts = comments_df.value_counts()
# get indices of those with value counts greater than or equal to 2
to_remove = value_counts[value_counts >2].index
# save df as df with all data filtered out
comments_df = comments_df[~comments_df.isin(to_remove)]

#check to make sure looks good
pd.DataFrame(comments_df).value_counts(ascending=False).reset_index()

sample_uncleaned = comments_df.copy()

In [225]:
sample_uncleaned.shape

(667937,)

In [226]:
#sample_uncleaned=pickle.load(open("sample_uncleaned.pickle",'rb'))

In [227]:
cryp_list=pickle.load(open("cryp_list_all_format.pickle", "rb"))

flat_list = [item for sublist in cryp_list for item in sublist]


## testing to filter on only a few cryptos to improve speed

In [228]:
sample_uncleaned=comments_df
a = np.array(cryp_list)
b = np.array([0,1,4,5,8,13,14,19])
cryp_list=a[b]


In [229]:
#only_uncleaned_dict #5k
#sample_uncleaned #1345964
#uncleaned_dict #75571


uncleaned_dict = {}
for index,cryp in enumerate(cryp_list):
    temp=pd.DataFrame(flat_list)
    temp=(temp[~temp[0].str.contains('|'.join(cryp_list[index]))][0]).tolist()
    temp=('\\b|\\b'.join(temp))
    uncleaned_dict[cryp[0]] = pd.DataFrame(sample_uncleaned[(sample_uncleaned.str.contains('\\b|\\b'.join(cryp_list[index])))&
                                                            (~sample_uncleaned.str.contains(temp))])

#get rid of empty keys
uncleaned_dict=uncleaned_dict={k:v for (k,v) in uncleaned_dict.items() if not v.empty}




/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [230]:
#list(uncleaned_dict['btc']['body'])

In [231]:
#pickle.dump(uncleaned_dict,open("only_uncleaned_dict.pickle", "wb" ))



In [232]:
#pd.concat(test.values(), ignore_index=True)
for crypto, df in uncleaned_dict.items():
    df['key'] = crypto
master_frame = pd.concat(uncleaned_dict, ignore_index=True)
#pd.DataFrame(master_frame)

#master_frame


In [233]:
master_frame['process']=list(nlp.pipe(master_frame['body'])) 
#master_frame

/opt/anaconda3/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [234]:
#test=test['btc'].sample(10)#.str.replace('\d+', '')
#test=test.map(lambda x: contractions.fix(x))
#test=pd.DataFrame(test.map(lambda x:re.sub(r'[^\w\s]','',x)))

#test=pd.DataFrame()


from spacy.lang.en.stop_words import STOP_WORDS
stop = STOP_WORDS
master_frame['adjnoun']=[[" ".join(token.lemma_.lower() for token in sentence if
           token.is_stop == False and 
           token.text.isalpha() == True and
           (token.pos_ == 'NOUN'or 
            token.pos_ =='ADJ') and 
           (len(token.lemma_)>=3 and len(token.text)<=15))] 
    for sentence in master_frame['process']]

master_frame['adj']=[[" ".join(token.lemma_.lower() for token in sentence if
           token.is_stop == False and 
           token.text.isalpha() == True and
           token.pos_ == 'ADJ' and 
           (len(token.lemma_)>=3 and len(token.text)<=15))] 
    for sentence in master_frame['process']]

master_frame['noun']=[[" ".join(token.lemma_.lower() for token in sentence if
           token.is_stop == False and 
           token.text.isalpha() == True and
           token.pos_ == 'NOUN' and 
           (len(token.lemma_)>=3 and len(token.text)<=15))] 
    for sentence in master_frame['process']]

In [250]:
#pickle.dump(master_frame,open("only_uncleaned_dict.pickle", "wb" ))



(55743, 6)

In [236]:
master_dict=dict(tuple(master_frame.groupby('key')))

In [237]:
#create dicts for the different pos of interest and convert contents into strings
adjnoun_dict=dict(tuple(master_frame.groupby('key')['adjnoun']))
noun_dict=dict(tuple(master_frame.groupby('key')['noun']))
adj_dict=dict(tuple(master_frame.groupby('key')['adj']))

#funcition to join lists in dict key dfs to strings
def try_join(l):
    try:
        return ','.join(map(str, l))
    except TypeError:
        return np.nan
    
#implementing fxn and removing empty values-not sure if this matters 
adjnoun_dict={key: list(filter(None,[try_join(sentence) for sentence in value])) for key, value in adjnoun_dict.items()}
noun_dict={key: list(filter(None,[try_join(sentence) for sentence in value])) for key, value in noun_dict.items()}
adj_dict={key: list(filter(None,[try_join(sentence) for sentence in value])) for key, value in adj_dict.items()}



In [238]:
#creating list of keys that have more than 1000 comments (in the adjnoun list)

key_list=list()
for key in adjnoun_dict:
    if len(adjnoun_dict[key])>100:
        key_list.append(key)
#key_list.remove('one')



adjnoun_dict_filtered={key: adjnoun_dict[key] for key in key_list}
noun_dict_filtered={key: noun_dict[key] for key in key_list}
adj_dict_filtered={key: adj_dict[key] for key in key_list}




### stopword list here is optional-consider using and modifying if larger amounts of data

In [322]:
from sklearn.feature_extraction import text 

# use these if using stemming instead of lemmatization
#from nltk.tokenize import WhitespaceTokenizer
#from nltk.tokenize import RegexpTokenizer, word_tokenize
#from nltk.stem import SnowballStemmer
#stemmer = SnowballStemmer("english")

#this should already be loaded in unless reading data from pickle
#import spacy
#nlp = spacy.load("en_core_web_trf")


#text.ENGLISH_STOP_WORDS.union(flat_list)

#read in again since we'll want to use ALL crypto names now
cryp_list=pickle.load(open("cryp_list_all_format.pickle", "rb"))

flat_list = [item for sublist in cryp_list for item in sublist]
#get common words across ALL cryptos-these are probably meaningless for crypto-specific topics
temp=('\\b|\\b'.join(flat_list))
#get top words 
cv = CountVectorizer(min_df=5)
#converting data to series containing all sentences to be use 
full_series=(pd.concat([pd.Series(v) for k,v in adjnoun_dict_filtered.items()]))
#transform 
X_cv_df=cv.fit_transform(full_series).toarray()
#add index, features
X_cv_df=pd.DataFrame(X_cv_df,index=full_series,columns=cv.get_feature_names())
#get word counts
word_counts=X_cv_df.sum().sort_values(ascending=False).reset_index()
#list flat list containing list of cryptos and now word counts greater than 1000
#the 1000 limit isn't necessary here since data was filtered more stringently-good to keep in for 
#future use
mask=list(word_counts[(word_counts['index'].str.contains(temp))|(word_counts[0]>1000)]['index'])
#removing nft from the list since it's a crypto name but a topic of interest-the word here
#refers to the topic and not the name of the unpopular crypto

#if stemming in preprocesing, use this after the above

flat_list=[word_tokenize(word) for word in flat_list]
#flat_list=[[stemmer.stem(word) for word in cryp] for cryp in flat_list]
flat_list = [item for sublist in flat_list for item in sublist]

#mask will only return values that we need to remove (eg if another crypto was in the flattened 
# cryp_list but not found in the data we have, it will NOT appear)
flat_list.extend(mask)
# extending list again found in the error "Tokenizing the stop words generated tokens"
flat_list.extend(['ai', 'axs', 'com', 'fetch', 'stacks', 'waves', 'yearn'])
flat_list.remove('nft')

#word_counts.head(25)

#joining to flat list to get lemmas
flat_list=" ".join(flat_list)

doc = nlp(flat_list)

flat_list=[token.lemma_ for token in doc]


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [240]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
import nltk

cv_tfidf = TfidfVectorizer(stop_words='english', max_df=.3, min_df=5)# strip_accents='ascii')#,max_df=0.9,min_df=.009)
#applying fit_transform on review
X_tfidf = cv_tfidf.fit_transform(adjnoun_dict_filtered['btc']).toarray()
#converting to df and naming columns and 
X_tf_df=pd.DataFrame(X_tfidf,index=adjnoun_dict_filtered['btc'],columns=cv_tfidf.get_feature_names())


nmf = NMF(6,init=None)
doc_topic = nmf.fit_transform(X_tf_df)
doc_topic_df=pd.DataFrame(doc_topic.round(5),
             index = adjnoun_dict_filtered['btc'])

X_tf_df.shape

/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


(17287, 1931)

In [241]:
topic_term=nmf.components_
topic_term_df=pd.DataFrame(topic_term.round(3), 
             #index = ["component_1", "component_2"],
             columns = cv_tfidf.get_feature_names())

topic_term_df

,ability,able,absolute,absurd,abundance,acceptance,access,accessible,accident,account,...,wrong,xbt,year,yesterday,yield,young,youth,youtuber,zero,zone
0,0.000,0.003,0.000,0.000,0.000,0.000,0.002,0.000,0.001,0.011,...,0.009,0.004,0.000,0.006,0.001,0.000,0.001,0.000,0.001,0.000
1,0.002,0.053,0.011,0.004,0.001,0.003,0.003,0.004,0.001,0.030,...,0.068,0.004,0.000,0.028,0.003,0.003,0.001,0.009,0.003,0.005
2,0.001,0.026,0.007,0.001,0.005,0.000,0.001,0.000,0.002,0.021,...,0.013,0.000,3.355,0.005,0.005,0.000,0.000,0.000,0.000,0.000
3,0.003,0.004,0.005,0.000,0.000,0.000,0.004,0.000,0.000,0.007,...,0.031,0.001,0.000,0.014,0.000,0.006,0.000,0.000,0.006,0.008
4,0.006,0.025,0.000,0.002,0.000,0.002,0.013,0.001,0.001,0.027,...,0.032,0.000,0.000,0.012,0.002,0.014,0.001,0.001,0.000,0.000
5,0.003,0.013,0.000,0.000,0.006,0.002,0.002,0.000,0.003,0.017,...,0.016,0.000,0.000,0.023,0.001,0.006,0.000,0.001,0.000,0.000


In [242]:
display_topics(nmf,cv_tfidf.get_feature_names(),6);




Topic  1
bitcoin, cash, king, moon, maxi, guy

Topic  2
market, coin, alt, price, bear, cap

Topic  3
year, end, month, worth, cycle, old

Topic  4
time, high, new, long, week, month

Topic  5
people, crypto, money, thing, lot, way

Topic  6
good, day, thing, today, guy, month




In [243]:

doc_topic_df=pd.DataFrame(doc_topic.round(5),
           index =adjnoun_dict_filtered['btc'])

#view sentences highest in topic 
pd.DataFrame(doc_topic_df[0].sort_values(ascending=False)).head(25)

,0
bitcoin,0.18935
bitcoin move hiking,0.18935
bitcoin,0.18935
bitcoin,0.18935
bitcoin,0.18935
bitcoin,0.18935
bitcoin,0.18935
bitcoin,0.18935
bitcoin,0.18935
bitcoin,0.18935


# Getting recommended number of components through turncated SVD explained ratios

In [244]:
#this funciton jsut needs the data and amount of variance wanted to capture

def tsvd_fxn(data, goal_var: float) -> int:
    tfv = TfidfVectorizer(stop_words='english',max_df=.2, min_df=5)#,ngram_range=(1, 3))
    #applying fit_transform on review
    X_tfidf = tfv.fit_transform(data).toarray()
    #converting to df and naming columns and 
    X_tf_df=pd.DataFrame(X_tfidf,index=data,columns=tfv.get_feature_names())
    #instantiate nmf
    lsa = TruncatedSVD(n_components=X_tf_df.shape[1]-1)
    #transform the X_vect_df 
    doc_topic = lsa.fit_transform(X_tf_df)
    #ratios explained
    var_ratio=lsa.explained_variance_ratio_
    # Create a function
    def select_n_components(var_ratio, goal_var: float) -> int:
        # Set initial variance explained so far
        total_variance = 0.0

        # Set initial number of features
        n_components = 0

        # For the explained variance of each feature:
        for explained_variance in var_ratio:

            # Add the explained variance to the total
            total_variance += explained_variance

            # Add one to the number of components
            n_components += 1

            # If we reach our goal level of explained variance
            if total_variance >= goal_var:
                # End the loop
                break

        # Return the number of components
        return(n_components)
    return(select_n_components(var_ratio, goal_var))


### Recommended number of components to account for .8 of variance

In [245]:
#adjectives and nouns
{key: tsvd_fxn(value,.8) for key, value in adjnoun_dict_filtered.items()}

{'ada': 328,
 'algo': 256,
 'btc': 657,
 'cro': 282,
 'dot': 177,
 'eth': 451,
 'matic': 146,
 'sol': 199}

In [246]:
#nouns only
{key: tsvd_fxn(value,.8) for key, value in noun_dict_filtered.items()}

{'ada': 229,
 'algo': 174,
 'btc': 479,
 'cro': 204,
 'dot': 124,
 'eth': 328,
 'matic': 104,
 'sol': 140}

In [247]:
#adjectives only
{key: tsvd_fxn(value,.8) for key, value in adj_dict_filtered.items()}

{'ada': 106,
 'algo': 85,
 'btc': 184,
 'cro': 81,
 'dot': 58,
 'eth': 126,
 'matic': 43,
 'sol': 62}

### Looking at stopwords and the number of words appearing most 

In [280]:
def nmf_fxn(data, dimensions, stop_words, num_words, max_df, min_df):
    dimensionlist = []
    for i in range(dimensions):
        dimensionlist.append("component_"+str(i+1))
    tfv = TfidfVectorizer(stop_words=stop_words,max_df=max_df, min_df=min_df)#,ngram_range=(1, 3))
    #applying fit_transform on review
    X_vect = tfv.fit_transform(data).toarray()
    #converting to df and naming columns and 
    X_vect_df=pd.DataFrame(X_vect,index=data,columns=tfv.get_feature_names())
    #instantiate nmf
    nmf = NMF(dimensions, init=None)
    #transform the X_vect_df 
    doc_topic = nmf.fit_transform(X_vect_df)
    # how much each term belongs to the different components
    topic_term=nmf.components_
    topic_term_df=pd.DataFrame(topic_term.round(3), 
             index =  dimensionlist,
             columns = tfv.get_feature_names())
    
    
    
    # most common word/phrase for each component
    feat_names = tfv.get_feature_names()
    word_dict = {};
    for i in range(dimensions):
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = nmf.components_[i].argsort()[:-num_words -1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['topic ' + '{:2d}'.format(i+1)] = words;
    topwords=pd.DataFrame(word_dict);


    
    # df for each term belongs to the different components
    doc_topic_df = pd.DataFrame(doc_topic.round(5),
             index = pd.Series(data),
             columns = dimensionlist)
  
    return(doc_topic_df, topic_term_df, topwords)



In [326]:
#a,b,c=nmf_fxn(adjnoun_dict_filtered['btc'],5,10)
#display(a,b,c)

for key in adjnoun_dict_filtered:
    a,b,c=nmf_fxn(adjnoun_dict_filtered[key],
                  dimensions=5,
                  stop_words=text.ENGLISH_STOP_WORDS.union(flat_list),
                  num_words=10,
                  max_df=.1,
                  min_df=5)
    print(key.upper())
    display(c.T)
    
#view sentences highest in score for indexed topic 
#pd.DataFrame(doc_topic_df[0].sort_values(ascending=False)).head(25)

ADA


,0,1,2,3,4,5,6,7,8,9
topic 1,stable,joke,meme,volatile,old,wrong,investment,nice,baby,bull
topic 2,bag,small,heavy,profit,volatility,stablecoin,huge,point,sure,future
topic 3,sub,sure,post,bad,shit,hate,wrong,opposite,little,shitcoin
topic 4,holder,smart,contract,guy,term,stablecoin,patience,past,future,shit
topic 5,pump,stablecoin,nice,little,post,guy,bit,hour,huge,reason


ALGO


,0,1,2,3,4,5,6,7,8,9
topic 1,governance,reward,staking,period,vote,official,apy,voting,option,poll
topic 2,moon,nice,meme,news,guy,solid,bit,faucet,ready,minute
topic 3,sub,meme,post,baby,love,favorite,bag,thank,wrong,official
topic 4,pump,governor,nice,reward,reason,fellow,hour,term,bit,option
topic 5,future,cheap,easy,bright,transaction,ecosystem,fast,nice,blockchain,low


BTC


...

,0,1,2,3,4,5,6,7,8,9
topic 1,ath,post,daily,low,chart,portfolio,ready,congrat,strong,pump
topic 2,killer,real,point,true,different,sub,low,wrong,term,centralized
topic 3,transaction,layer,solution,cheap,chain,exchange,low,problem,blockchain,fast
topic 4,end,guy,problem,world,possible,road,post,prediction,bull,solid
topic 5,moon,dip,profit,sure,bad,bag,alt,nice,guy,portfolio


MATIC


,0,1,2,3,4,5,6,7,8,9
topic 1,bag,solid,guy,moon,news,small,profit,rollup,happy,shit
topic 2,pump,huge,sure,bit,nice,little,ath,love,ready,action
topic 3,automatic,buy,exchange,profit,game,weekly,dca,process,staking,ass
topic 4,dramatic,bit,dip,point,sub,news,guy,little,exchange,bad
topic 5,cheap,transaction,chain,fast,low,moon,cap,rollup,solution,choice


SOL


,0,1,2,3,4,5,6,7,8,9
topic 1,fud,holder,point,post,old,supply,conference,subreddit,ath,amazing
topic 2,sub,post,bad,hate,wrong,moon,sentiment,shit,opinion,dead
topic 3,centralized,shit,decentralized,blockchain,validator,node,exchange,team,bank,shitcoin
topic 4,news,old,bad,post,shit,sure,team,current,true,dev
topic 5,bag,dip,holder,pump,glad,guy,small,profit,little,happy


## deepmoji stuff

In [330]:
#conda install pytorch -c pytorch
#! pip install text-unidecode
#! pip install emoji --upgrade
#! pip install -U scikit-learn

Obtaining file:///Users/ClaytonYoung/Library/Mobile%20Documents/com~apple~CloudDocs/coding/Unsupervised/Topic%20Modeling
ERROR: file:///Users/ClaytonYoung/Library/Mobile%20Documents/com~apple~CloudDocs/coding/Unsupervised/Topic%20Modeling does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [345]:
!python scripts/download_weights.py

python: can't open file 'scripts/download_weights.py': [Errno 2] No such file or directory


In [342]:
!python scripts/download_weights.py

python: can't open file 'scripts/download_weights.py': [Errno 2] No such file or directory
